# Problem recommendation

# Import Modules

In [1]:
import warnings, random
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


from itertools import permutations # For making pairs

warnings.filterwarnings('ignore')

# Data loading

In [2]:
user_problem = pd.read_csv('./userId_problemId.csv', usecols=['userId', 'problemId', 'level', 'title'])
problem_category = pd.read_csv('./problem_category.csv', usecols=['problemId', 'category']) # for title-matching

In [3]:
print(user_problem.head())
print(problem_category.head())

    userId  problemId  level      title
0  sos0911       1000      1        A+B
1  sos0911       1001      1        A-B
2  sos0911       1002      7         터렛
3  sos0911       1003      8    피보나치 함수
4  sos0911       1005     13  ACM Craft
   problemId        category
0       1000      arithmetic
1       1000  implementation
2       1000            math
3       1001      arithmetic
4       1001  implementation


# Preprocessing data


1. user_problem으로 부터 골드 이상의 문제를 기록.
2. 사용자 마다 푼 문제 목록을 $(problem1, problem2)$, $(problem2, problem1)$ 과 같이 pair로 생성.
3. 2번 과정이 끝난 후, random을 이용해 각 pair 순서를 무작위로 shuffle.

In [4]:
#문제 level 10 이상만 사용 (브론즈, 실버 문제 사용 x, 골드 이상만 사용)
user_problem = user_problem[user_problem['level'] > 10]

#실제로 유저가 푼 문제의 목록화(골드 이상 문제)
unique_problem = user_problem['problemId'].unique()

#푼 문제 개수
problem_size=len(unique_problem)
print(user_problem.shape)
print("problem count:", problem_size)

#골드 이상의 문제들 빼고는 다 드랍
problem_category = problem_category[problem_category['problemId'].isin(unique_problem)]

(279500, 4)
problem count: 2755


In [5]:
#problems = selected_problems['problemId'].unique()
selected_problems = user_problem

#user별로 저장
users = dict()
users_pairs = dict()

cur_user = -1
user_prob = []
for row in selected_problems.itertuples():
    if cur_user != row.userId and cur_user != -1:
        users[cur_user] = user_prob
        user_prob = [row.problemId]
    else :
        user_prob.append(row.problemId)
    cur_user = row.userId
users[cur_user] = user_prob

#5%만 shuffle 후 사용
# for i in users.keys():
#     random.shuffle(users[i])
#     users[i] = users[i][0:len(users[i])*5//100]

# accuracy를 위한 pair 생성 키-problemId 값-[problemId로 예측하면 나와도 되는 결과problemId들]
accuracy_pair = dict()
for i in users.keys():
    for k in range(len(users[i])):
        if  users[i][k] in accuracy_pair:
            accuracy_pair[users[i][k]] = accuracy_pair[users[i][k]] + users[i][:k]+users[i][k+1:]
        else:
            accuracy_pair[users[i][k]] = users[i][:k]+users[i][k+1:]

#중복 쌍 제거
for i in accuracy_pair.keys():
    accuracy_pair[i] = np.unique(accuracy_pair[i])
    
#pair 생성
for i in users.keys():
    users_pairs[i] = torch.combinations(torch.tensor(np.unique(users[i])))

#pair쌍 합치기
problem_pairs = torch.tensor([])
for i in users_pairs.keys():
    problem_pairs = torch.cat((problem_pairs, users_pairs[i]), dim=0)

#pair의 반대 부분도 만들기
problem_pairs = torch.cat((problem_pairs, torch.flip(problem_pairs, dims=[0, 1])), dim=0)

#shuffle
random.shuffle(problem_pairs)

In [8]:
print(problem_pairs.shape)

torch.Size([42219262, 2])


# Build and train neural networks for generating problem embeddings


- 각 문제 임베딩을 구하기 위해 뉴럴 네트워크 모델을 활용하여 multi-class classification 을 수행

- 설계할 신경망의 기본 구조는 **Input Layer - Hidden(Embedding) Layer - Output Layer**.
    - 7주차 강의자료 p.7 신경망 구조 이미지 참고

- 현재 Network를 통해 하고자 하는 task는 multi-class classification.
    - 예: $(problem1, problem2)$ 와 같은 입력 데이터와 정답 출력 데이터를 이용해 모델을 학습
        - Input : $problem1$의 one-hot vector
        - Output : $\widehat{problem2}$의 one-hot vector
        - Compute Loss : $\widehat{problem2}$ 와 $problem2$ 간의 Cross-entropy Loss
- 학습이 완료된 이후에 input layer와 hidden(embedding) layer 사이의 weight matrix $W_{in}$를 problem에 대한 embedding vector로 사용이 가능.

- 설계한 뉴럴 네트워크 모델의 학습이 완료된 후, 학습된 weight matrix $W_{in}$의 행/열벡터를 각 영화에 대한 embedding vector로 간주하여 문제 embedding 들을 구할 수 있음.

In [10]:
#training set, test set분리
train_set = problem_pairs[:len(problem_pairs)//10*9]
test_set = problem_pairs[len(problem_pairs)//10*9:]

problems = selected_problems['problemId'].unique()

#one hot encoding
oneHot = pd.get_dummies(problems)

#gpu 사용을 위해 cuda
# torch.cuda.empty_cache()
# device = "cuda" if torch.cuda.is_available() else "cpu"
# cuda = torch.device(device)

In [11]:
#problem id와 one hot mapping할 dict만들기
problem_map = dict()
problem_index_to_id = dict()
index = 0
for i in sorted(problems):
    problem_map[i] = index
    problem_index_to_id[index] = i
    index += 1

In [12]:
#Custom Dataset
class CustomDataset(Dataset):
    def __init__(self, raw_data):
        self.problems = raw_data

    def __len__(self):
        return len(self.problems)

    def __getitem__(self, idx):
        return torch.tensor(oneHot[self.problems[idx][0].item()], dtype=torch.float32), torch.tensor(problem_map[self.problems[idx][1].item()]), self.problems[idx][0].item()

In [13]:
#model 생성
class EmbeddingModel(nn.Module):
    def __init__(self):
        super(EmbeddingModel, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(oneHot.shape[0], 40),
            nn.ReLU(),
            nn.Linear(40, oneHot.shape[0]),
        )
        
    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [14]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y, pId) in enumerate(dataloader):
        # 예측(prediction)과 손실(loss) 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y, pId in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            pred_pid = pred.argmax(dim=1)
            
            for i in range(len(pId)):
                #pred_pid는 index이기 때문에 다시 문제 id로 변환 해줘야 함.
                correct += int(problem_index_to_id[pred_pid[i].item()] in accuracy_pair[int(pId[i])])

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [15]:
model = EmbeddingModel()

# 파라미터 설정
learning_rate = 0.01
beta1 = 0.9
beta2 = 0.999
batch_size = 256
epochs = 5
# 옵티마이저 초기화
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(beta1, beta2))
#loss 초기화
loss_fn = nn.CrossEntropyLoss()
#기존에는 batch size = 64
train_dataloader = DataLoader(CustomDataset(train_set.to()), batch_size=batch_size, shuffle=False,drop_last=False)
test_dataloader = DataLoader(CustomDataset(test_set.to()), batch_size=batch_size, shuffle=False,drop_last=False)

#학습
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 7.928399  [    0/37997334]
loss: 5.530487  [25600/37997334]
loss: 5.776050  [51200/37997334]
loss: 5.786795  [76800/37997334]
loss: 5.879909  [102400/37997334]
loss: 5.948178  [128000/37997334]
loss: 6.081445  [153600/37997334]
loss: 6.166691  [179200/37997334]
loss: 6.009717  [204800/37997334]
loss: 6.000907  [230400/37997334]
loss: 6.002212  [256000/37997334]
loss: 6.081055  [281600/37997334]
loss: 6.167660  [307200/37997334]
loss: 6.169698  [332800/37997334]
loss: 6.127420  [358400/37997334]
loss: 6.092095  [384000/37997334]
loss: 6.092077  [409600/37997334]
loss: 6.265545  [435200/37997334]
loss: 6.241042  [460800/37997334]
loss: 6.088000  [486400/37997334]
loss: 6.147972  [512000/37997334]
loss: 6.248278  [537600/37997334]
loss: 6.295578  [563200/37997334]
loss: 6.222786  [588800/37997334]
loss: 6.033552  [614400/37997334]
loss: 6.203891  [640000/37997334]
loss: 6.196033  [665600/37997334]
loss: 6.199835  [691200/37997334]
loss: 6.2745

# Recommend customized problems to user


- 임의의 한명의 사용자에 대하여 해당 사용자가 풀었던 문제 n개에 대해 **통합된 embedding vector**를 생성.
    - n개의 embedding vector들에 대해, element-wise한 계산을 통해 통합된 하나의 embedding vector를 생성.
    - 이 embedding vector는 해당 사용자의 전반적인 영화 시청 성향을 나타내는 embedding vector로 간주할 수 있음.
    - 즉, **사용자 1명 당 1개의 embedding vector**를 가짐.
- 통합된 embedding vector와 학습된 weight matrix $W_{in}$의 모든 영화 embedding vector들 간의 유사도를 계산.
- 그 중 유사도가 높은 (top n) 영화들을 선정, 사용자에게 추천.

In [16]:
#category 합치기
categories = pd.DataFrame(columns = ['problemId', 'category'])
category_sum = ""
cur_problem = -1
for row in problem_category.itertuples():
    if cur_problem != row.problemId and cur_problem != -1:
        categories = categories.append({'problemId': cur_problem, 'category': category_sum}, ignore_index=True)
        category_sum = row.category
    else:
        category_sum = category_sum + "/" + str.strip(row.category)
    cur_problem = row.problemId
categories = categories.append({'problemId': row.problemId, 'category': category_sum}, ignore_index=True)
print(categories.head())
#join
total_info = user_problem.join(categories.set_index('problemId'), on='problemId')
total_info.head()

#문제 정보 매핑용
problem_info = dict()
currentProblemId = -1
for row in total_info.itertuples():
    if not row.problemId in problem_info.keys():
        problem_info[row.problemId] = (row.title, row.category, row.level)
        currentProblemId = row.problemId

  problemId                        category
0      1005  /dp/graphs/topological_sorting
1      1006                              dp
2      1007               bruteforcing/math
3      1011                            math
4      1013                    regex/string


In [17]:
#cosine 유사도를 사용하여 풀지않은 상위 몇개의 문제를 추천
def recommendProblem(user_id, user_embedding, problem_embedding, problem_list, recommend_count = 5):
    print('user {0}'.format(user_id))
    cos = nn.CosineSimilarity(dim=0, eps=1e-6)
    res = []
    for i in problem_embedding.keys():
        if not i in problem_list:
            res.append((cos(problem_embedding[i], user_embedding[user_id]), i, problem_info[i][0], problem_info[i][1], problem_info[i][2]))
    res = sorted(res, reverse=True)[0:recommend_count]
    
    length = recommend_count if len(res) > recommend_count else len(res)
    for i in range(length):
        print('problemId: {0}, title: {1}, category: {2}, level: {3}, Similarity: {4:0.4f}'.format(res[i][1], res[i][2], res[i][3], res[i][4], res[i][0]))
    print("")
    
    return res

In [18]:
#각 영화별로 embedding 생성
problem_embedding = dict()
for param in model.parameters():
    for i in range(param.shape[1]):
        problem_embedding[problem_index_to_id[i]] = param[:,i]
    break

In [19]:
#사용자에 대한 embedding 생성
user_embedding = dict()
for i in users.keys():
    user_embedding[i] = torch.tensor(np.zeros(len(problem_embedding[1005])))
    for k in users[i]:
        user_embedding[i] += problem_embedding[k]
    user_embedding[i] /= len(users[i])

In [20]:
#문제 embedding과 유사도 계산 및 추천
recommend = dict()
recommend_count = 5
for i in users.keys():
    recommend[i] = recommendProblem(i, user_embedding, problem_embedding, users[i], recommend_count)

user sos0911
problemId: 2595, title: 배수, category: bfs/graph_traversal/graphs/math/number_theory, level: 20, Similarity: 0.9529
problemId: 9426, title: 중앙값 측정, category: data_structures/segtree, level: 16, Similarity: 0.9460
problemId: 4181, title: Convex Hull, category: convex_hull/geometry/sorting, level: 16, Similarity: 0.9442
problemId: 9343, title: 랜덤 걷기, category: combinatorics/flt/math/modular_multiplicative_inverse/number_theory, level: 15, Similarity: 0.9435
problemId: 3172, title: 현주와 윤주의 재미있는 단어 게임, category: data_structures/segtree/sorting/string, level: 18, Similarity: 0.9427

user dnfl182
problemId: 2176, title: 합리적인 이동경로, category: dijkstra/dp/graph_traversal/graphs, level: 14, Similarity: 0.9246
problemId: 2171, title: 직사각형의 개수, category: bruteforcing/coordinate_compression, level: 12, Similarity: 0.9206
problemId: 3172, title: 현주와 윤주의 재미있는 단어 게임, category: data_structures/segtree/sorting/string, level: 18, Similarity: 0.9139
problemId: 7578, title: 공장, category: data_s

2595번 등의 문제는 sos0911 사용자가 풀어보지 않은 문제

In [21]:
#weight값 저장
torch.save(model, "./model.pt")
torch.save(model.state_dict,"./model_state_dict.pt")
torch.save({
    'model':model.state_dict(),
    'optimizer':optimizer.state_dict()
},'all.tar')

# problem Embedding으로 원래 문제 맞춰보기

문제 embedding과 유저 embedding의 유사도가 높은 상위 100개의 문제가 원래 유저가 푼 문제와 얼마나 겹치는지를 확인해봤다.

In [22]:
def predictProblem(user_id, user_embedding, problem_embedding, problem_list, recommend_count = 5):
    cos = nn.CosineSimilarity(dim=0, eps=1e-6)
    res = []
    for i in problem_embedding.keys():
        res.append((cos(problem_embedding[i], user_embedding[user_id]), i))
    res = sorted(res, reverse=True)[0:recommend_count]
    
    count = 0
    for i in res:
        if i[1] in problem_list:
            count += 1
    count = count / len(res) * 100
    print('user: {0}, accuracy: {1}'.format(user_id, count))
    
    return count

user_accuracy = dict()
recommend_count = 100
for i in users.keys():
    user_accuracy[i] = predictProblem(i, user_embedding, problem_embedding, users[i], recommend_count)
    
accuracy_mean = 0
for i in user_accuracy.keys():
    accuracy_mean += user_accuracy[i]
print(accuracy_mean/len(user_accuracy.keys()))

user: sos0911, accuracy: 5.0
user: dnfl182, accuracy: 4.0
user: hja5432, accuracy: 12.0
user: jhnan917, accuracy: 3.0
user: siontama, accuracy: 2.0
user: wjd2632, accuracy: 4.0
user: woongseob12, accuracy: 6.0
user: deserve82, accuracy: 5.0
user: jeonyj1020, accuracy: 7.000000000000001
user: ljoghe, accuracy: 16.0
user: overwatcher123, accuracy: 11.0
user: rlwjd4177, accuracy: 10.0
user: sandlekang, accuracy: 6.0
user: whdauddbs, accuracy: 6.0
user: an3735297, accuracy: 11.0
user: ensia96, accuracy: 7.000000000000001
user: c4big, accuracy: 9.0
user: dozzing729, accuracy: 3.0
user: hsunj2003, accuracy: 9.0
user: object1997, accuracy: 4.0
user: qye5856, accuracy: 10.0
user: jayjangjooho, accuracy: 10.0
user: kangjoseph90, accuracy: 13.0
user: nw4611, accuracy: 10.0
user: 2556, accuracy: 4.0
user: as00098, accuracy: 16.0
user: chad6027, accuracy: 7.000000000000001
user: geon0407, accuracy: 4.0
user: logeon0725, accuracy: 6.0
user: ten_ability, accuracy: 9.0
user: thoughtman2, accuracy: 4.

user: tkdgus0782, accuracy: 6.0
user: chkun3109, accuracy: 3.0
user: chobe1, accuracy: 8.0
user: girawhale, accuracy: 12.0
user: give654, accuracy: 6.0
user: heung, accuracy: 6.0
user: hyunya, accuracy: 5.0
user: js9028, accuracy: 4.0
user: jsm3465, accuracy: 11.0
user: sylim123, accuracy: 9.0
user: vmffotltka, accuracy: 4.0
user: dsa2341, accuracy: 8.0
user: lkyeon328, accuracy: 9.0
user: lovenody, accuracy: 12.0
user: qoalstjr, accuracy: 7.000000000000001
user: alswns5607, accuracy: 8.0
user: hws0728, accuracy: 4.0
user: khcho902, accuracy: 10.0
user: nain95, accuracy: 7.000000000000001
user: pdj9696, accuracy: 5.0
user: tjdgur23, accuracy: 7.000000000000001
user: ybs1164, accuracy: 8.0
user: 112ckek, accuracy: 4.0
user: 97k07h11, accuracy: 6.0
user: rkdtkdtn0706, accuracy: 5.0
user: gmroh06, accuracy: 4.0
user: hhhh1352, accuracy: 5.0
user: iron1209, accuracy: 6.0
user: smk6221, accuracy: 8.0
user: thereon42, accuracy: 8.0
user: beaverbae, accuracy: 14.000000000000002
user: hjl9345,

user: naangel0515, accuracy: 5.0
user: skyyy6, accuracy: 9.0
user: wjdgur9534, accuracy: 6.0
user: dlgldgld, accuracy: 12.0
user: guriguri8833, accuracy: 6.0
user: ju_h2, accuracy: 5.0
user: kimdh425, accuracy: 7.000000000000001
user: proqk, accuracy: 5.0
user: zozond, accuracy: 5.0
user: adung7, accuracy: 7.000000000000001
user: dndkwk, accuracy: 4.0
user: dndyd1231, accuracy: 9.0
user: jaehoo1, accuracy: 5.0
user: kimyh0316, accuracy: 4.0
user: rangsuk, accuracy: 8.0
user: rlawngus0910, accuracy: 10.0
user: selfm, accuracy: 4.0
user: dnrkgjsrn, accuracy: 9.0
user: dudgns2, accuracy: 5.0
user: gw0419, accuracy: 2.0
user: rlarlghks970113, accuracy: 3.0
user: staphaniek, accuracy: 2.0
user: ychh11232, accuracy: 9.0
user: choijoohee, accuracy: 7.000000000000001
user: gi1dong2, accuracy: 4.0
user: h9661, accuracy: 8.0
user: jangjeon, accuracy: 6.0
user: ljw8541, accuracy: 11.0
user: yuhanbae06, accuracy: 5.0
user: awayfromkeyboard, accuracy: 8.0
user: bkwak, accuracy: 4.0
user: bsa0322, a

user: kyskkm2537, accuracy: 10.0
user: mindong7046, accuracy: 9.0
user: shikshik99, accuracy: 6.0
user: simm4256, accuracy: 4.0
user: thoon916, accuracy: 6.0
user: chrisais9, accuracy: 3.0
user: dgk051215, accuracy: 5.0
user: hueguem, accuracy: 9.0
user: kgh1030, accuracy: 6.0
user: marshmelloooooooooow, accuracy: 1.0
user: qorwnseop3, accuracy: 12.0
user: riverlike14, accuracy: 8.0
user: shin421179, accuracy: 6.0
user: sun950903, accuracy: 9.0
user: wlsdndml213, accuracy: 2.0
user: ywywyw, accuracy: 7.000000000000001
user: zoomspeed, accuracy: 5.0
user: cjeongmin, accuracy: 6.0
user: expect017, accuracy: 4.0
user: gm27376, accuracy: 4.0
user: jinsj1, accuracy: 5.0
user: joon8409, accuracy: 5.0
user: kimkh7534, accuracy: 6.0
user: kimmulgae, accuracy: 4.0
user: rain7i, accuracy: 3.0
user: reach1994, accuracy: 7.000000000000001
user: zox2323, accuracy: 6.0
user: bae1776, accuracy: 3.0
user: cheongm, accuracy: 7.000000000000001
user: duno72, accuracy: 5.0
user: ejqmf94, accuracy: 5.0
use

user: jdyun15, accuracy: 4.0
user: wodhks1105, accuracy: 9.0
user: yhs03043, accuracy: 6.0
user: yjsmk0902, accuracy: 3.0
user: cleankid99, accuracy: 3.0
user: good1317, accuracy: 2.0
user: blessmealways00, accuracy: 3.0
user: gunny6026, accuracy: 6.0
user: l_hyun, accuracy: 5.0
user: powerwonjun, accuracy: 2.0
user: r4bb1t, accuracy: 7.000000000000001
user: rollony, accuracy: 5.0
user: zizon5941, accuracy: 11.0
user: aigorithm, accuracy: 7.000000000000001
user: bogun11904, accuracy: 8.0
user: honeysleep, accuracy: 6.0
user: kkt219a, accuracy: 4.0
user: miki308, accuracy: 4.0
user: minhookang, accuracy: 10.0
user: newsanghoon, accuracy: 5.0
user: okcho9807, accuracy: 6.0
user: qhtjd844, accuracy: 3.0
user: qkrehddus01, accuracy: 5.0
user: sane2710, accuracy: 6.0
user: sy46, accuracy: 8.0
user: wandukong, accuracy: 5.0
user: yeveyn, accuracy: 7.000000000000001
user: yyoouunngg, accuracy: 4.0
user: hun6728, accuracy: 9.0
user: jane0516, accuracy: 5.0
user: minjungw00, accuracy: 6.0
user:

user: orangewhitefox, accuracy: 4.0
user: x75201, accuracy: 4.0
user: yslim6168, accuracy: 7.000000000000001
user: aspresso, accuracy: 2.0
user: bluejoyq, accuracy: 2.0
user: pjkov0824, accuracy: 5.0
user: tang1993, accuracy: 3.0
user: 1sss123ss, accuracy: 2.0
user: abel1802, accuracy: 6.0
user: dong149, accuracy: 8.0
user: nmrhtn7898, accuracy: 1.0
user: sabsari5, accuracy: 4.0
user: adds136078, accuracy: 12.0
user: bc7817, accuracy: 4.0
user: bds193, accuracy: 5.0
user: beawall, accuracy: 3.0
user: ju214425, accuracy: 7.000000000000001
user: kasiwagi_hikari, accuracy: 4.0
user: kdhae1231, accuracy: 6.0
user: psm7335, accuracy: 5.0
user: qwerty7525, accuracy: 7.000000000000001
user: zpdlswmf12, accuracy: 13.0
user: alonso14, accuracy: 8.0
user: dhdhkeh1, accuracy: 11.0
user: dktlsk6, accuracy: 3.0
user: dygks232, accuracy: 7.000000000000001
user: eachotherbest, accuracy: 7.000000000000001
user: eeeuns, accuracy: 3.0
user: gangjeuk, accuracy: 5.0
user: gse123, accuracy: 3.0
user: gustj

user: ap4o, accuracy: 7.000000000000001
user: chongal0128, accuracy: 6.0
user: connor101, accuracy: 2.0
user: jaee07, accuracy: 3.0
user: jse03310, accuracy: 3.0
user: nis9857, accuracy: 4.0
user: snmhz325, accuracy: 2.0
user: vecum0814, accuracy: 2.0
user: orb_h, accuracy: 4.0
user: riverboy94, accuracy: 3.0
user: wts1597, accuracy: 3.0
user: yujong_lee, accuracy: 4.0
user: 4100jh, accuracy: 7.000000000000001
user: gbds234, accuracy: 3.0
user: krong6930, accuracy: 5.0
user: llsg822, accuracy: 0.0
user: saksak1527, accuracy: 7.000000000000001
user: renkousami, accuracy: 4.0
user: silverlic, accuracy: 2.0
user: suddenly, accuracy: 6.0
user: vionicbest, accuracy: 5.0
user: yimjunhyuck, accuracy: 4.0
user: 314programs, accuracy: 3.0
user: asdf99245, accuracy: 6.0
user: cocobino, accuracy: 5.0
user: comolove, accuracy: 6.0
user: geniemo, accuracy: 4.0
user: gun2841, accuracy: 4.0
user: jaeminkim408, accuracy: 3.0
user: kinh5026, accuracy: 6.0
user: oolop, accuracy: 7.000000000000001
user: 

user: hogisim314, accuracy: 1.0
user: hyunwoo045, accuracy: 4.0
user: jo1997, accuracy: 7.000000000000001
user: layer6ai, accuracy: 8.0
user: leek018, accuracy: 6.0
user: marona, accuracy: 6.0
user: yellow2041, accuracy: 4.0
user: 95kim1, accuracy: 4.0
user: chewin9, accuracy: 3.0
user: chotjd329, accuracy: 6.0
user: kn1996, accuracy: 4.0
user: kyo289, accuracy: 6.0
user: mini0shark, accuracy: 6.0
user: pmqa, accuracy: 3.0
user: yoyoyo5639, accuracy: 5.0
user: hishine6, accuracy: 2.0
user: presentsong, accuracy: 3.0
user: qkrwnghd96, accuracy: 4.0
user: sobu0715, accuracy: 4.0
user: boxer1532, accuracy: 4.0
user: exito, accuracy: 3.0
user: kimmysh, accuracy: 5.0
user: liging, accuracy: 2.0
user: ruddks1001, accuracy: 5.0
user: wnsrl4067, accuracy: 5.0
user: yms21888, accuracy: 5.0
user: a891, accuracy: 2.0
user: emforhs246, accuracy: 8.0
user: gkscodus11, accuracy: 4.0
user: gsmin2020, accuracy: 5.0
user: j961224, accuracy: 3.0
user: kimug2145, accuracy: 6.0
user: nanta0112, accuracy: 